## SKLEARN



### Imports and read in data

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
import unicodedata
import re
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

from dateutil.relativedelta import relativedelta

In [ ]:
import pyLDAvis.sklearn
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import MiniBatchKMeans

In [ ]:
with open('dataframes\\final_wa_df.pickle', 'rb') as handle:
    all_wa_df = pickle.load(handle)

with open('dataframes\\final_stan_df.pickle', 'rb') as handle:
    all_stan_df = pickle.load(handle)


wa_content = all_wa_df.content
stan_content = all_stan_df.content

In [5]:
all_wa_df

,filename,title,date,category,content,embeddings
10112,wa936bik6.txt,Minder kinderen geboren,2013-01-08,België,Minder kinderen geboren In Vlaamse ziekenhuize...,"[0.8717681169509888, 0.21637214720249176, 0.67..."
10113,wa936bik7.txt,Vlaams medicijn tegen tbc,2013-01-08,België,Vlaams medicijn tegen tbc Onderzoekers van het...,"[-0.1551680862903595, 0.6744857430458069, 0.13..."
10114,wa936bik8.txt,Directrice verdacht van spieken,2013-01-08,België,Directrice verdacht van spieken De directrice ...,"[-0.3453265130519867, -0.008550132624804974, -..."
10118,wa936bin3.txt,Weinig vrouwen met sjerp in ons land,2013-01-08,België,Weinig vrouwen met sjerp in ons land In januar...,"[-0.2693834602832794, 0.9814454913139343, -0.1..."
10116,wa936bin1.txt,Noémie is Miss België,2013-01-08,België,Noémie is Miss België Noémie Happart is de moo...,"[-0.3678942322731018, -0.24943530559539795, -0..."
...,...,...,...,...,...,...
6084,wa1164bui5.txt,Duizenden kleine pinguïns dood,2017-11-16,Buitenland,Duizenden kleine pinguïns dood Een grote ramp ...,"[-0.21590065956115723, 0.23220542073249817, -0..."
6085,wa1164buk1.txt,Nog meer uitstel in Congo,2017-11-16,Buitenland,Nog meer uitstel in Congo In Congo komen er ge...,"[-1.5455697774887085, -0.9892030358314514, 0.0..."
6086,wa1164buk2.txt,Nieuwe regering in Nederland,2017-11-16,Buitenland,Nieuwe regering in Nederland Nederland heeft e...,"[-1.0460810661315918, -0.7230560779571533, 0.5..."
6088,wa1164idk1.txt,"Ik wil onrecht aanklagen, maar liefst met een ...",2017-11-16,In de kijker,"""Ik wil onrecht aanklagen, maar liefst met een...","[-0.40642863512039185, -0.12022947520017624, 0..."


In [5]:
#Drop NaNs

all_wa_df = all_wa_df.dropna(subset=['title'])
all_stan_df = all_stan_df.dropna(subset=['title'])

all_wa_df = all_wa_df[all_wa_df.title != 'No title']
all_stan_df = all_stan_df[all_stan_df.title != 'No title']

### Preprocessing

In [6]:
#preprocessing functions

def remove_accents(text):
    text= unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def remove_special_characters(text, remove_digits=False):

    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'

    text = re.sub(pattern, '',text)

    return text

stopword_list = nltk.corpus.stopwords.words('dutch')

def remove_stopwords(text, lower_case=False):
    tokens = nltk.word_tokenize(text, language='dutch')
    tokens = [token.strip().strip("'").lower() for token in tokens]

    if lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]

    #turn it back into 1 string to give as input to the sklearn vectorizer
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

In [8]:
all_wa_df['title_no_stop'] = all_wa_df['title'].apply(lambda x: remove_accents(x))
all_wa_df['title_no_stop'] = all_wa_df['title_no_stop'].apply(lambda x: remove_special_characters(x))
all_wa_df['title_no_stop'] = all_wa_df['title_no_stop'].apply(lambda x: remove_stopwords(x, True))

all_stan_df['title_no_stop'] = all_stan_df['title'].apply(lambda x: remove_accents(x))
all_stan_df['title_no_stop'] = all_stan_df['title_no_stop'].apply(lambda x: remove_special_characters(x))
all_stan_df['title_no_stop'] = all_stan_df['title_no_stop'].apply(lambda x: remove_stopwords(x, True))

In [8]:
all_wa_df['content_no_stop'] = all_wa_df['content'].apply(lambda x: remove_accents(x))
all_wa_df['content_no_stop'] = all_wa_df['content_no_stop'].apply(lambda x: remove_special_characters(x))
all_wa_df['content_no_stop'] = all_wa_df['content_no_stop'].apply(lambda x: remove_stopwords(x, True))

all_stan_df['content_no_stop'] = all_stan_df['content'].apply(lambda x: remove_accents(x))
all_stan_df['content_no_stop'] = all_stan_df['content_no_stop'].apply(lambda x: remove_special_characters(x))
all_stan_df['content_no_stop'] = all_stan_df['content_no_stop'].apply(lambda x: remove_stopwords(x, True))

In [9]:
all_wa_df["Newspaper"] = "Wablieft"
all_stan_df["Newspaper"] = "Standaard"

In [13]:
all_df = pd.concat([all_wa_df, all_stan_df])

In [28]:
# with open('allPreprocessedDf.pickle', 'wb') as handle:
#     pickle.dump(all_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('dataframes\\allPreprocessedDf.pickle', 'rb') as handle:
    all_df = pickle.load(handle)

In [5]:
all_content = all_df["content_no_stop"].tolist()

### Tf-IDF Titles


In [9]:
wa_titles = all_wa_df["title_no_stop"].tolist()

vector = TfidfVectorizer(max_df=0.3,
                             stop_words=stopword_list, 
                             lowercase=True,
                             use_idf=True,
                             norm=u'l2',
                             smooth_idf=True
                            )
tfidf = vector.fit_transform(wa_titles)

In [10]:
with open('models\\tfidfVectorizer.pickle', 'wb') as handle:
    pickle.dump(vector, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
stan_titles = all_stan_df["title_no_stop"].tolist()

stan_transformed = vector.transform(stan_titles)

In [22]:
with open('dataframes\manualRatedDf.pickle', 'rb') as handle:
    manualRatedDf = pickle.load(handle)

In [23]:
manualRatedDf = manualRatedDf.dropna(subset=['WaTitle'])
manualRatedDf = manualRatedDf.dropna(subset=['StanTitle'])

manualRatedDf = manualRatedDf[manualRatedDf.WaTitle != 'No title']
manualRatedDf = manualRatedDf[manualRatedDf.StanTitle != 'No title']

In [28]:
intermediateDf = pd.DataFrame()

intermediateDf['WaTitleNoStop'] = manualRatedDf['WaTitle'].apply(lambda x: remove_accents(x))
intermediateDf['WaTitleNoStop'] = intermediateDf['WaTitleNoStop'].apply(lambda x: remove_special_characters(x))
intermediateDf['WaTitleNoStop'] = intermediateDf['WaTitleNoStop'].apply(lambda x: remove_stopwords(x, True))

intermediateDf['StanTitleNoStop'] = manualRatedDf['StanTitle'].apply(lambda x: remove_accents(x))
intermediateDf['StanTitleNoStop'] = intermediateDf['StanTitleNoStop'].apply(lambda x: remove_special_characters(x))
intermediateDf['StanTitleNoStop'] = intermediateDf['StanTitleNoStop'].apply(lambda x: remove_stopwords(x, True))

In [29]:
tfidfSimList = []
waTitleList = intermediateDf["WaTitleNoStop"].tolist()
stanTitleList = intermediateDf["StanTitleNoStop"].tolist()

for i in range(0, len(waTitleList)):
    wa_tfidf = vector.transform([waTitleList[i]])
    stan_tfidf = vector.transform([stanTitleList[i]])
    tfidfSimList.append(linear_kernel(wa_tfidf, stan_tfidf))

NameError: name 'vector' is not defined

In [57]:
simList = [simScore[0][0] for simScore in tfidfSimList]

In [61]:
manualRatedDf["tfidfTitle"] = simList

In [93]:
with open('dataframes\classifierDf.pickle', 'wb') as handle:
    pickle.dump(manualRatedDf, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Add titletfidf to first evaluated articles

In [10]:
with open('firstEvalDf.pickle', 'rb') as handle:
    firstEvalDf = pickle.load(handle)

In [19]:
waFileList = firstEvalDf.WaFile.tolist()
stanFileList = firstEvalDf.StanFile.tolist()
stanFileList= [ele.tolist()[0] for ele in stanFileList]

In [23]:
test1=all_wa_df.loc[all_wa_df["filename"] == waFileList[0]].title_no_stop.tolist()[0]
test2=all_stan_df.loc[all_stan_df["filename"] == stanFileList[0]].title_no_stop.tolist()[0]

In [26]:
print(linear_kernel(vector.transform([test1]), vector.transform([test2])))

[[0.]]


In [28]:
tfidfSimList = []
for i in range(0, len(waFileList)):
    wa_title = all_wa_df.loc[all_wa_df["filename"] == waFileList[i]].title_no_stop.tolist()[0]
    stan_title = all_stan_df.loc[all_stan_df["filename"] == stanFileList[i]].title_no_stop.tolist()[0]
    tfidfSimList.append(linear_kernel(vector.transform([wa_title]), vector.transform([stan_title])))

In [30]:
simList = [simScore[0][0] for simScore in tfidfSimList]

In [32]:
firstEvalDf["tfidfTitle"] = simList

In [33]:
with open('firstEvalDf.pickle', 'wb') as handle:
    pickle.dump(firstEvalDf, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Vectorize Content


In [5]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                          for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [7]:
vectorizer = TfidfVectorizer(max_df=0.90, min_df=20, max_features=5000)

tfidf_matrix = vectorizer.fit_transform(all_content)

In [8]:
with open('tfidfFitAllContent.pickle', 'wb') as handle:
    pickle.dump(vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
feature_names = vectorizer.get_feature_names()
doc_term_matrix_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=list(feature_names))

,01,02,090035987,10,100,1000,10000,100000,11,12,...,zware,zwart,zwarte,zweden,zweedse,zwembad,zwemmen,zwijgen,zwitserland,zwitserse
0,0.0,0.0,0.0,0.0,0.0,0.147796,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184483,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184484,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184485,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
184486,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
doc_term_matrix_tfidf.head()

In [12]:
lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', max_iter=500, random_state=0).fit(tfidf_matrix)

In [8]:
with open('LDAmodel.pickle', 'wb') as handle:
    pickle.dump(lda_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('LDAmodel.pickle', 'rb') as handle:
    lda_model = pickle.load(handle)

In [9]:
lda_matrix = lda_model.transform(tfidf_matrix)

In [13]:
with open('allTfidfContentMatrix.pickle', 'wb') as handle:
    pickle.dump(tfidf_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('allLDAMatrix.pickle', 'wb') as handle:
    pickle.dump(lda_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
testlist = lda_matrix.tolist()

In [20]:
all_df["LDA"] = lda_matrix.tolist()

In [24]:
tfidf_list = tfidf_matrix.tolist()

AttributeError: tolist not found

In [26]:
all_df["titleTFIDF"] = tfidf_matrix

In [21]:
display_topics(lda_model, feature_names, 15)

Topic 0:
we jaar zegt euro wel nieuwe procent mensen moeten onze twee gaat brussel miljoen waar
Topic 1:
veroordeeld cel rechtbank gevangenis rechter boete bedraagt maanden celstraf straf beringen focus kim beroep euro
Topic 2:
onderwijs leerlingen school studenten scholen leuven google leuvense ouders kinderen leerkrachten baby schauvliege beiden joke
Topic 3:
brand uur verkeer brandweer ongeval wagen mobiliteit politie auto nmbs rijden brug fietsers werkzaamheden bestuurder
Topic 4:
graden griekse oekraine wind nv delhaize heverlee opklaringen 3001 meteoservices 090035987 weerlijn wwwmeteoservicesbe kapeldreef maxima
Topic 5:
club anderlecht seizoen wedstrijd ploeg voetbal finale wk spelers ronde league spelen won rode standard
Topic 6:
politie man parket zone opgepakt verdachte agenten onderzoek slachtoffer verdacht ingediend verdachten dader vrouw onderzoeksrechter
Topic 7:
partij trump regering verkiezingen nva president premier parlement politieke europese partijen ps stemmen pol

In [16]:
vectorizer = TfidfVectorizer(max_df=0.90, min_df=100, max_features=5000, use_idf=True)

tfidf_stan_matrix = vectorizer.fit_transform(all_stan_df.content_no_stop)

In [17]:
lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', max_iter=500, random_state=0).fit(tfidf_stan_matrix)

In [18]:
with open('stan_lda_model.pickle', 'wb') as handle:
    pickle.dump(lda_model, handle, protocol=pickle.HIGHEST_PROTOCOL)